In [1]:
# import libriaries
import sqlite3

In [2]:
connection = sqlite3.connect("chinook.db")
cursor = connection.cursor()

In [30]:
# 1. Which tracks appeared in the most playlists? How many playlist did they appear in?Which track generated the most revenue?
cursor.execute ('''WITH first_query AS (
                SELECT TrackId, COUNT(*) AS Quantity, PlaylistId
                FROM playlist_track
                GROUP BY 3
                ORDER BY 2 DESC
                ), 
                second_query AS (
                SELECT SUM (Quantity) AS 'Total Number', TrackId
                FROM first_query
                GROUP BY 2
                ORDER BY 1 DESC
                ), 
                third_query AS (
                SELECT *
                FROM second_query
                LEFT JOIN tracks
                    ON second_query.TrackId = tracks.TrackId
                )
                SELECT "Total Number", "TrackId", "Name", "Composer"
                FROM third_query;''').fetchall()

[(6606,
  1,
  'For Those About To Rock (We Salute You)',
  'Angus Young, Malcolm Young, Brian Johnson'),
 (1477,
  3,
  'Fast As a Shark',
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (426, 2819, 'Battlestar Galactica: The Story So Far', None),
 (100, 3403, 'Intoitus: Adorate Deum', 'Anonymous'),
 (39, 215, 'Sozinho', 'Peninha'),
 (25, 3479, 'Prometheus Overture, Op. 43', 'Ludwig van Beethoven'),
 (25,
  3430,
  'Toccata and Fugue in D Minor, BWV 565: I. Toccata',
  'Johann Sebastian Bach'),
 (15, 52, 'Man In The Box', 'Jerry Cantrell, Layne Staley'),
 (1, 3402, 'Band Members Discuss Tracks from "Revelations"', None),
 (1, 597, "Now's The Time", 'Miles Davis')]

In [7]:
# 2. Which track generated the most revenue?
cursor.execute ('''WITH first_query AS (
                SELECT SUM(UnitPrice) AS "Revenue by Track", TrackId
                FROM invoice_items
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 10
                ), 
                second_query AS (
                SELECT *
                FROM first_query
                LEFT JOIN tracks
                    ON first_query.TrackId = tracks.TrackId
                )
                SELECT "TrackId", "Name", "Revenue by Track"
                FROM second_query;''').fetchone()

(2832, 'The Woman King', 3.98)

In [8]:
# 3. Which album generated the most revenue?
cursor.execute ('''WITH first_query AS (
                SELECT *
                FROM invoice_items
                LEFT JOIN tracks
                    ON invoice_items.TrackId = tracks.TrackId
                ), 
                second_query AS (
                SELECT SUM (UnitPrice) AS "Revenue by Album", AlbumId
                FROM first_query
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 10
                ), 
                third_query AS (
                SELECT *
                FROM second_query
                LEFT JOIN albums
                    ON second_query.AlbumId= albums.AlbumId
                )
                SELECT "AlbumId", "Title", "ArtistId", "Revenue by Album"
                FROM third_query;''').fetchone()

(253, 'Battlestar Galactica (Classic), Season 1', 158, 35.81999999999999)

In [9]:
# 4. Which genre generated the most revenue?
cursor.execute ('''WITH first_query AS (
                SELECT *
                FROM invoice_items
                LEFT JOIN tracks
                    ON invoice_items.TrackId = tracks.TrackId
                ), 
                second_query AS (
                SELECT ROUND (SUM (UnitPrice),2) AS "Revenue by Genre", GenreId
                FROM first_query
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 10
                ), 
                third_query AS (
                SELECT *
                FROM second_query
                LEFT JOIN genres
                    ON second_query.GenreId = genres.GenreId
                )
                SELECT GenreId, Name, "Revenue by Genre"
                FROM third_query;''').fetchone()

(1, 'Rock', 826.65)

In [10]:
# 5. Which countries have the highest sales revenue?
cursor.execute ('''SELECT ROUND (SUM (Total),0) AS "Sales Revenue by Country", BillingCountry
                FROM invoices
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 5;''').fetchall()

[(523.0, 'USA'),
 (304.0, 'Canada'),
 (195.0, 'France'),
 (190.0, 'Brazil'),
 (156.0, 'Germany')]

In [28]:
# 6. What percent of total revenue does each country make up?
cursor.execute ('''WITH first_query AS (
                SELECT ROUND (SUM (Total),0) AS "Sales Revenue by Country", BillingCountry
                FROM invoices
                GROUP BY 2
                ORDER BY 1 DESC
                )
                SELECT BillingCountry, "Sales Revenue by Country", 
                       (ROUND(SUM("Sales Revenue by Country") / (SELECT SUM("Sales Revenue by Country") FROM first_query), 3))*100 AS Percentage
                FROM first_query
                GROUP BY BillingCountry
                ORDER BY percentage DESC;''').fetchall()

[('USA', 523.0, 22.400000000000002),
 ('Canada', 304.0, 13.0),
 ('France', 195.0, 8.4),
 ('Brazil', 190.0, 8.1),
 ('Germany', 156.0, 6.7),
 ('United Kingdom', 113.0, 4.8),
 ('Czech Republic', 90.0, 3.9),
 ('Portugal', 77.0, 3.3000000000000003),
 ('India', 75.0, 3.2),
 ('Ireland', 46.0, 2.0),
 ('Hungary', 46.0, 2.0),
 ('Chile', 47.0, 2.0),
 ('Netherlands', 41.0, 1.7999999999999998),
 ('Finland', 42.0, 1.7999999999999998),
 ('Austria', 43.0, 1.7999999999999998),
 ('Sweden', 39.0, 1.7000000000000002),
 ('Norway', 40.0, 1.7000000000000002),
 ('Spain', 38.0, 1.6),
 ('Poland', 38.0, 1.6),
 ('Italy', 38.0, 1.6),
 ('Denmark', 38.0, 1.6),
 ('Belgium', 38.0, 1.6),
 ('Australia', 38.0, 1.6),
 ('Argentina', 38.0, 1.6)]

In [29]:
# 7. How many customers did each employee support?
cursor.execute ('''WITH first_query AS(SELECT *
                FROM customers
                LEFT JOIN employees
                    ON customers.SupportRepId = employees.EmployeeId
                )
                SELECT COUNT (*) AS "Total Number", EmployeeId, "LastName:1" AS "Last Name", "FirstName:1" AS "First Name", Title
                FROM first_query
                GROUP BY EmployeeId;''').fetchall()

[(21, 3, 'Peacock', 'Jane', 'Sales Support Agent'),
 (20, 4, 'Park', 'Margaret', 'Sales Support Agent'),
 (18, 5, 'Johnson', 'Steve', 'Sales Support Agent')]

In [37]:
# 8. What is the average revenue for each sale?
cursor.execute ('''WITH first_query AS (
                SELECT CustomerId, ROUND(AVG (Total), 2) AS "Average Sale"
                FROM invoices
                GROUP BY CustomerId
                ORDER BY 2 DESC
                ), 
                second_query AS(
                SELECT *
                FROM first_query
                LEFT JOIN customers
                    ON first_query.CustomerId = customers.CustomerId
                ), 
                third_query AS (
                SELECT ROUND(SUM ("Average Sale"), 2) AS "Average Revenue per Sale", SupportRepId
                FROM second_query
                GROUP BY SupportRepId
                )
                SELECT EmployeeId, LastName, FirstName, "Average Revenue per Sale"
                FROM third_query
                LEFT JOIN employees
                    ON third_query.SupportRepId = employees.EmployeeId
                ORDER BY "Average Sale by Employee" DESC;''').fetchall()

[(3, 'Peacock', 'Jane', 119.86),
 (4, 'Park', 'Margaret', 110.72),
 (5, 'Johnson', 'Steve', 102.84)]

In [38]:
# 9. What is their total sale?
cursor.execute ('''WITH first_query AS (
                SELECT CustomerId, ROUND(SUM (Total),0) AS "Total Revenue"
                FROM invoices
                GROUP BY CustomerId
                ORDER BY 2 DESC
                ), 
                second_query AS(
                SELECT *
                FROM first_query
                LEFT JOIN customers
                    ON first_query.CustomerId = customers.CustomerId
                ), 
                third_query AS (
                SELECT SUM ("Total Revenue") AS "Total Revenue per Employee", SupportRepId
                FROM second_query
                GROUP BY SupportRepId
                )
                SELECT EmployeeId, LastName, FirstName, "Total Revenue per Employee"
                FROM third_query
                LEFT JOIN employees
                    ON third_query.SupportRepId = employees.EmployeeId
                ORDER BY "Total Revenue by Employee" DESC;''').fetchall()

[(3, 'Peacock', 'Jane', 841.0),
 (4, 'Park', 'Margaret', 783.0),
 (5, 'Johnson', 'Steve', 727.0)]

In [48]:
# 10. Do longer or shorter length albums tend to generate more revenue?
cursor.execute ('''WITH first_query AS (
                SELECT *
                FROM invoice_items
                LEFT JOIN tracks
                    ON invoice_items.TrackId = tracks.TrackId
                ), 
                second_query AS (
                SELECT ROUND(SUM (UnitPrice),2) AS "Revenue by Album", AlbumId
                FROM first_query
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 10
                ), 
                third_query AS (
                SELECT *
                FROM second_query
                LEFT JOIN albums
                    ON second_query.AlbumId= albums.AlbumId
                ), 
                revenue_by_album AS (
                SELECT "AlbumId", "Title", "ArtistId", "Revenue by Album"
                FROM third_query
                ), 
                number_of_tracks AS (
                SELECT COUNT (*) AS "Number of Tracks", AlbumId, Name
                FROM tracks
                GROUP BY AlbumId
                ORDER BY 1 DESC
                ), 
                join_table AS (
                SELECT *
                FROM revenue_by_album
                LEFT JOIN number_of_tracks
                    ON revenue_by_album.AlbumId = number_of_tracks.AlbumId
                )
                SELECT AlbumId, Title, "Number of Tracks", "Revenue by Album"
                FROM join_table
                ORDER BY 4 DESC;''').fetchall()

# Medium length albums tend to generate more revenue.

[(253, 'Battlestar Galactica (Classic), Season 1', 24, 35.82),
 (251, 'The Office, Season 3', 25, 31.84),
 (23, 'Minha Historia', 34, 26.73),
 (231, 'Lost, Season 2', 24, 25.87),
 (228, 'Heroes, Season 1', 23, 25.87),
 (141, 'Greatest Hits', 57, 25.74),
 (73, 'Unplugged', 30, 24.75),
 (227, 'Battlestar Galactica, Season 3', 19, 23.88),
 (229, 'Lost, Season 3', 26, 21.89),
 (224, 'Acústico', 22, 21.78)]

In [50]:
# 11. Is the number of times a track appear in any playlist a good indicator of sales?
cursor.execute ('''WITH first_query AS (
                SELECT SUM(UnitPrice) AS "Revenue by Track", TrackId
                FROM invoice_items
                GROUP BY 2
                ORDER BY 1 DESC
                ), 
                second_query AS (
                SELECT *
                FROM first_query
                LEFT JOIN tracks
                    ON first_query.TrackId = tracks.TrackId
                ), 
                revenue_by_track AS(
                SELECT "Revenue by Track", "TrackId", "Name"
                FROM second_query
                ), 
                number_of_appearance AS (
                SELECT COUNT (*) AS "Number of Appearance", TrackId
                FROM playlist_track
                GROUP BY TrackId
                ORDER BY 1 DESC
                )
                SELECT *
                FROM number_of_appearance 
                LEFT JOIN revenue_by_track
                    ON number_of_appearance.TrackId = revenue_by_track.TrackId
                WHERE "Revenue by Track" IS NOT NULL
                ORDER BY "Revenue by Track" DESC;''').fetchmany(500)

# Absolutely not.

[(2, 2832, 3.98, 2832, 'The Woman King'),
 (2, 2850, 3.98, 2850, 'The Fix'),
 (2, 2868, 3.98, 2868, 'Walkabout'),
 (2, 3177, 3.98, 3177, 'Hot Girl'),
 (2, 3200, 3.98, 3200, 'Gay Witch Hunt'),
 (2, 3214, 3.98, 3214, "Phyllis's Wedding"),
 (2, 3223, 3.98, 3223, 'How to Stop an Exploding Man'),
 (2, 3250, 3.98, 3250, 'Pilot'),
 (2, 2820, 1.99, 2820, 'Occupation / Precipice'),
 (2, 2821, 1.99, 2821, 'Exodus, Pt. 1'),
 (2, 2822, 1.99, 2822, 'Exodus, Pt. 2'),
 (2, 2823, 1.99, 2823, 'Collaborators'),
 (2, 2826, 1.99, 2826, 'Hero'),
 (2, 2827, 1.99, 2827, 'Unfinished Business'),
 (2, 2828, 1.99, 2828, 'The Passage'),
 (2, 2833, 1.99, 2833, 'A Day In the Life'),
 (2, 2837, 1.99, 2837, 'Crossroads, Pt. 1'),
 (2, 2838, 1.99, 2838, 'Crossroads, Pt. 2'),
 (2, 2839, 1.99, 2839, 'Genesis'),
 (2, 2841, 1.99, 2841, 'One Giant Leap'),
 (2, 2844, 1.99, 2844, 'Better Halves'),
 (2, 2845, 1.99, 2845, 'Nothing to Hide'),
 (2, 2846, 1.99, 2846, 'Seven Minutes to Midnight'),
 (2, 2851, 1.99, 2851, 'Distractio

In [59]:
# 12. How much revenue is generated each year, and what is its percent change from the previous year?
cursor.execute ('''WITH Revenue AS (
                SELECT ROUND(SUM (Total), 2) AS Revenue, CAST(strftime('%Y', InvoiceDate) AS INTEGER) AS "Current Year", CAST(strftime('%Y', InvoiceDate)-1 AS INTEGER) AS "Previous Year"
                FROM invoices
                GROUP BY 2
                ORDER BY 2
                )
                SELECT current."Previous Year", current."Current Year", current.Revenue, ROUND((current.Revenue - previous.Revenue)/previous.Revenue * 100, 2) AS "% change compared to the last year"
                FROM Revenue current 
                LEFT JOIN Revenue previous
                    ON current."Previous Year" = previous."Current Year";''').fetchall()

[(2008, 2009, 449.46, None),
 (2009, 2010, 481.45, 7.12),
 (2010, 2011, 469.58, -2.47),
 (2011, 2012, 477.53, 1.69),
 (2012, 2013, 450.58, -5.64)]

In [ ]:
# Commit changes to DB and close connections
cursor.close()
connection.commit()
connection.close()